In [1]:
import pandas as pd 
data_train = pd.read_csv('/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task1_Tamil/tamil_offensive_train.tsv', sep = '\t')
data_test = pd.read_csv('/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task1_Tamil/tam_offesive_withoutlabels_test.tsv', sep = '\t')

In [2]:
data_train.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [3]:
data_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [4]:
data_test.shape

(654, 2)

In [5]:
data_train.shape

(5880, 3)

In [6]:
data_train['category'].value_counts()

NOT          4724
OFF          1153
not-Tamil       3
Name: category, dtype: int64

In [7]:
data_train_final = data_train[data_train['category'] != 'not-Tamil']

In [8]:
data_train_final.shape

(5877, 3)

In [9]:
data_train_final['category'].value_counts()

NOT    4724
OFF    1153
Name: category, dtype: int64

In [10]:
from sklearn import preprocessing
import numpy as np

le = preprocessing.LabelEncoder()

le.fit(data_train_final['category'])
print(le.classes_)

y_train_index = le.transform(data_train_final['category'])
#y_test_index = le.transform(label_test)


['NOT' 'OFF']


In [11]:
from keras.utils.np_utils import to_categorical
y_train_cal  =  to_categorical(y_train_index)


In [12]:
y_train_index

array([0, 1, 0, ..., 0, 1, 0])

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data_train_final['text'], y_train_cal, test_size = 0.2, random_state=42)

In [14]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape 

((4701,), (1176,), (4701, 2), (1176, 2))

For word = ngram_range (1,3)
For char = ngram_range (1,6)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec = TfidfVectorizer(ngram_range=(1,6), analyzer = 'char_wb', max_features = 30000)
#tfidfvec = TfidfVectorizer(ngram_range=(1,3), max_features = 50000)
tfidf_train = tfidfvec.fit_transform(x_train)
tfidf_val = tfidfvec.transform(x_val)
tfidf_test = tfidfvec.transform(data_test['text'])


In [16]:
#tfidfvec.get_feature_names()

In [17]:
tfidf_train.shape

(4701, 30000)

In [18]:
x_train = tfidf_train
x_test = tfidf_val
y_train = y_train
y_test = y_val

In [23]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(4096, input_dim = tfidf_train.shape[1], activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 4096)              122884096 
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 130       
Total params: 125,014,722
Trainable params: 125,014,722
Non-trainable params: 0
________________________________________

In [24]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(x_train.toarray(), y_train, validation_data=(x_test.toarray(), y_test), epochs=10, batch_size=20, verbose=2 )

Epoch 1/10
236/236 - 7s - loss: 0.3810 - accuracy: 0.8154 - val_loss: 0.3433 - val_accuracy: 0.8333
Epoch 2/10
236/236 - 6s - loss: 0.2293 - accuracy: 0.8981 - val_loss: 0.3692 - val_accuracy: 0.8299
Epoch 3/10
236/236 - 6s - loss: 0.0984 - accuracy: 0.9619 - val_loss: 0.5359 - val_accuracy: 0.8367
Epoch 4/10
236/236 - 6s - loss: 0.0318 - accuracy: 0.9894 - val_loss: 0.8202 - val_accuracy: 0.8163
Epoch 5/10
236/236 - 6s - loss: 0.0164 - accuracy: 0.9945 - val_loss: 0.9407 - val_accuracy: 0.8274
Epoch 6/10
236/236 - 6s - loss: 0.0076 - accuracy: 0.9972 - val_loss: 1.1863 - val_accuracy: 0.8240
Epoch 7/10
236/236 - 6s - loss: 0.0139 - accuracy: 0.9960 - val_loss: 0.9840 - val_accuracy: 0.8299
Epoch 8/10
236/236 - 6s - loss: 0.0111 - accuracy: 0.9964 - val_loss: 1.1964 - val_accuracy: 0.8231
Epoch 9/10
236/236 - 6s - loss: 0.0062 - accuracy: 0.9972 - val_loss: 1.1432 - val_accuracy: 0.8257
Epoch 10/10
236/236 - 6s - loss: 0.0048 - accuracy: 0.9981 - val_loss: 1.4079 - val_accuracy: 0.8197

In [31]:
predictions_test_submission = model.predict(tfidf_test.toarray())

In [32]:
predictions_test_submission

array([[1.0000000e+00, 3.8582321e-21],
       [1.0000000e+00, 1.6337608e-19],
       [1.0000000e+00, 2.2210453e-29],
       ...,
       [4.7476737e-16, 1.0000000e+00],
       [1.0000000e+00, 1.4369438e-22],
       [2.1753272e-02, 9.7824681e-01]], dtype=float32)

In [33]:
import numpy as np
predictions_test_submission = np.asarray(predictions_test_submission)

In [34]:
import numpy as np
predictions_test_submission1 = np.zeros_like(predictions_test_submission)
predictions_test_submission1[np.arange(len(predictions_test_submission)), predictions_test_submission.argmax(1)] = 1

In [35]:
predictions_test_submission1

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [36]:
predictions_test_submission1_final = np.argmax(predictions_test_submission1, axis=1)

In [37]:
predictions_test_submission1_final1 = le.inverse_transform(predictions_test_submission1_final)

In [38]:
data_test['label'] = predictions_test_submission1_final1

In [39]:
data_test

,id,text,label
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [40]:
data_test.to_csv("/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task1_Tamil/ANU_Task1_Tamil.tsv", sep="\t", index=False)

In [41]:
check_submission = pd.read_csv("/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task1_Tamil/ANU_Task1_Tamil.tsv", sep="\t")

In [42]:
check_submission

,id,text,label
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT
